## Assignment 2 - Part 2 - Finding Twitter Influencers

### Top 100 Influencers

In [2]:
import pandas as pd
import tweepy

consumer_key = 'vZyOIzO8AZBgJ6U2DfjvyN9hi'
consumer_secret = 'jy8HKcp8OkQmacmyccnhaAq06e3BDBblas0f2ub0jWrRhVqan8'
access_token_key = '829374717941448704-eFTxdEdwJG67jNr8fI4vQEaCHSn7Css'
access_token_secret = '2kXFyEbXFhhi9bJNhlDpYHwak7798PhnU52Dx2YI2cak5'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api=tweepy.API(auth)

#### Pull 5,000 Tweets from Twitter API

In [56]:
query = 'Trump'
count = 100
max_tweets = 5000
searched_tweets = []
last_id = -1
while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break

#### Mine Tweets for Relevant Information

In [88]:
trump_tweet_data = []

for tweet in searched_tweets: #Create a list of lists of the attributes of tweets
    k=[]
    k = k+[tweet.author.screen_name,tweet.author.listed_count,tweet.author.followers_count,tweet.text
           ,tweet.retweet_count,tweet.favorite_count,tweet.in_reply_to_screen_name]
    
    # If a retweet object exists, pull in user name of retweet. Otherwise, add blank entry.
    if hasattr(tweet, 'retweeted_status'):
        k.append(tweet.retweeted_status.user.screen_name)
    else: k.append('')
        
    # Pull in and convert Mentions object to a list of usernames
    mention_list = []
    for user_dict in tweet.entities['user_mentions']:
        if user_dict['screen_name'] != k[-1] and user_dict['screen_name'] != k[-2]: # Fix so that retweets and replies are not also included as mentions
            mention_list.append(user_dict['screen_name'])
            
    k.append(mention_list)
    
    trump_tweet_data.append(k)
print trump_tweet_data[:5] 

[[u'SherriKuczeryk', 23, 119, u'RT @JordanUhl: admit it, Trump, you want state-run media: https://t.co/Mgua3G0fIQ', 508, 0, None, u'JordanUhl', []], [u'RexRedbone', 39, 470, u'RT @infowars: Plan to save humanity UNDER ATTACK!! &amp; #Grammy awards given go-ahead to bash #Trump - Tune in LIVE 4pm-6pm CT: https://t.co/F\u2026', 46, 0, None, u'infowars', []], [u'SamaraLouzi', 16, 4380, u'Pilot removed from flight after Donald Trump rant over intercom before take off https://t.co/JttwICJk6O', 0, 0, None, '', []], [u'elpatron_145', 3, 298, u'RT @SenSanders: I disagreed with President Bush all the time. I never called him a pathological liar. He was just conservative. But Trump l\u2026', 29017, 0, None, u'SenSanders', []], [u'Daggard', 18, 343, u'RT @Breznican: BREAKING: Trump advisor Stephen Miller confirms it rubs the lotion on its skin or else it gets the hose again. https://t.co/\u2026', 4380, 0, None, u'Breznican', []], [u'3perPatriot', 1, 66, u'RT @billboard: Joy Villa unveils Donald T

#### Convert to Dataframe

In [110]:
trump_tweet_data_df = pd.DataFrame(trump_tweet_data) #Convert it into a dataframe and publish a csv file
trump_tweet_data_df.columns = ['User','listed_count','followers_count','text','retweets',
                               'favorites','Reply_User','Retweet_User','Mention_Users']

trump_tweet_data_df.head()

,User,listed_count,followers_count,text,retweets,favorites,Reply_User,Retweet_User,Mention_Users
0,SherriKuczeryk,23,119,"RT @JordanUhl: admit it, Trump, you want state...",508,0,None,JordanUhl,[]
1,RexRedbone,39,470,RT @infowars: Plan to save humanity UNDER ATTA...,46,0,None,infowars,[]
2,SamaraLouzi,16,4380,Pilot removed from flight after Donald Trump r...,0,0,None,,[]
3,elpatron_145,3,298,RT @SenSanders: I disagreed with President Bus...,29017,0,None,SenSanders,[]
4,Daggard,18,343,RT @Breznican: BREAKING: Trump advisor Stephen...,4380,0,None,Breznican,[]


#### Define Attribute Weights

In [92]:
listed_count_weight = 0.44/(0.44+0.40+0.28)  #Calculate weights based on their importances
follower_count_weight = 0.40/(0.44+0.40+0.28)
retweet_count_weight = 0.28/(0.44+0.40+0.28)

#### Pre-processing

Remove unnecessary columns, keep only unique users, and normalize list, follower & retweet counts

In [111]:
from sklearn import preprocessing #Scale the data and score it

top_influencers_df = trump_tweet_data_df[['User','listed_count','followers_count','retweets']].drop_duplicates()

top_influencers_df['listed_norm'] = preprocessing.scale(top_influencers_df['listed_count'])
top_influencers_df['followers_norm'] = preprocessing.scale(top_influencers_df['followers_count'])
top_influencers_df['retweets_norm'] = preprocessing.scale(top_influencers_df['retweets'])

#### Calculated Weighted User Influence Score

In [116]:
top_influencers_df['Score'] = top_influencers_df['listed_norm']*listed_count_weight\
+top_influencers_df['followers_norm']*follower_count_weight\
+top_influencers_df['retweets_norm']*retweet_count_weight

top100influencers_trump = top_influencers_df.sort('Score',ascending=False)[:100]
top100influencers_trump[:10]

C:\Users\Dallas\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,User,listed_count,followers_count,retweets,listed_norm,followers_norm,retweets_norm,Score
1170,htTweets,5951,5048757,1,11.757003,50.554688,-0.269788,22.606621
974,thehill,20015,1947355,4,39.959096,19.463365,-0.269404,22.582067
1495,24HorasTVN,7668,3092590,1,15.200048,30.944260,-0.269788,16.955522
446,darrenrovell,11472,1695674,26245,22.828089,16.940281,3.090865,15.790994
2872,AlbertoRavell,11353,1715397,2,22.589462,17.138003,-0.269660,14.927732
3733,teleSURtv,10989,1374561,0,21.859544,13.721148,-0.269916,13.420609
660,dailykos,7693,247580,3,15.250180,2.423247,-0.269532,6.789205
2245,LOLGOP,6891,322574,14,13.641955,3.175057,-0.268123,6.426257
528,FlorianDny,2,80,192595,-0.172338,-0.057922,24.392671,6.009777
2955,ChannelNewsAsia,4107,735461,1,8.059288,7.314218,-0.269788,5.710923


In [94]:
#trump_df_unique.sort('Score',ascending=False)[:100].to_csv('top100.csv',encoding='utf-8')

C:\Users\Dallas\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


## Influence Matrix

#### Create new DF with only relevant columns

##### Key Assumptions:
1. Their should be a reference line EACH TIME a user retweets another user (i.e. if a user retweets another user twice, their will be two entries)
2. Retweets and replies should not also count as mentions (i.e. if a user retweets a user, there will be an entry for just the retweet rathen then 1 for the retweet AND 1 for the mention).

In [117]:
influence_matrix = trump_tweet_data_df[['User','Reply_User','Retweet_User','Mention_Users']]
influence_matrix.head()

,User,Reply_User,Retweet_User,Mention_Users
0,SherriKuczeryk,None,JordanUhl,[]
1,RexRedbone,None,infowars,[]
2,SamaraLouzi,None,,[]
3,elpatron_145,None,SenSanders,[]
4,Daggard,None,Breznican,[]
